<a href="https://colab.research.google.com/github/odailsono2/CHdocenteIFRNjc/blob/main/Dim_Engrenagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# @title Fator q
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Exibição no Colab
import IPython.display as display

# Dados da imagem organizados em DataFrames
dados_externo = {
    "n_dentes": [10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 24, 28, 34, 40, 50, 65, 80, 100],
    "fator_q": [5.2, 4.9, 4.5, 4.3, 4.1, 3.9, 3.7, 3.6, 3.5, 3.3, 3.2, 3.1, 3.0, 2.9, 2.8, 2.7, 2.6, 2.6]
}
df_externo = pd.DataFrame(dados_externo)

dados_interno = {
    "n_dentes": [20, 24, 30, 38, 50, 70, 100, 200],
    "fator_q": [1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5][:8]  # limitado a 8 valores
}
df_interno = pd.DataFrame(dados_interno)

# Função de interpolação
def interpolar_q(n_dentes, tipo='externo'):
    if tipo == 'externo':
        df = df_externo
    elif tipo == 'interno':
        df = df_interno
    else:
        raise ValueError("Tipo inválido: escolha 'externo' ou 'interno'.")

    interp_func = interp1d(df['n_dentes'], df['fator_q'], kind='linear', fill_value="extrapolate")
    return float(interp_func(n_dentes))

# # Exemplos de uso:
# print(interpolar_q(31, tipo='externo'))  # interpolação externa
# print(interpolar_q(31, tipo='interno'))  # interpolação interna

In [15]:
# @title Tabela de Dureza
import pandas as pd

###Carrega Tabela de Dureza

# Substitua pelo seu ID real da planilha
sheet_id = "1GmkBQOGUkKs6RmJMdjNqkbrkxzXkzLpPP5s2Znt4gRo"
sheet_name = "Tabela_Completa_de_Convers_o_de_Dureza"  # ou "Sheet1", "Planilha1" etc.

# Gera o link de exportação CSV
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carrega no DataFrame
df_dureza = pd.read_csv(csv_url)

# Definindo os títulos das colunas

# # Visualiza
# import pandas as pd
# from IPython.display import display
# display(df)


def converter_dureza(valor, de: str, para: str):
    """
    Converte um valor de dureza de uma escala para outra,
    usando o valor mais próximo da tabela, ignorando valores NaN.
    """
    if de not in df_dureza.columns or para not in df_dureza.columns:
        return f"Coluna '{de}' ou '{para}' inválida."

    # Remove linhas com NaN na coluna de origem
    dados_validos = df_dureza.dropna(subset=[de])

    # Seleciona a linha com menor diferença para o valor informado
    linha_mais_proxima = dados_validos.iloc[(dados_validos[de] - valor).abs().argsort()[:1]]

    resultado = linha_mais_proxima.iloc[0][para]

    return float(resultado) if pd.notna(resultado) else "Sem correspondência disponível"


# converter_dureza(60,"Rockwell_C", "Dureza_HB")

In [16]:
# @title Carrega Lista de Tensão de Materiais
# Substitua pelo seu ID real da planilha
sheet_id = "1oD-ugKvYCNAVJXnOAn-SwyeQto2QKA6pRr6565umGdQ"
sheet_name = "materiaisTensaoPA"  # ou "Sheet1", "Planilha1" etc.
# https://docs.google.com/spreadsheets/d/1oD-ugKvYCNAVJXnOAn-SwyeQto2QKA6pRr6565umGdQ/edit?usp=sharing
# Gera o link de exportação CSV
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carrega no DataFrame
df_materiais = pd.read_csv(csv_url, decimal=',')

## Exibe Dataframe
# from IPython.display import display
# display(df_materiais)


# Função de consulta
def consultar_tensao_material(material: str):
    linha = df_materiais[df_materiais["Material"].str.contains(material, case=False, na=False)]
    if linha.empty:
        return f"Material '{material}' não encontrado."
    return linha.iloc[0]["Tensao_MPa"]

print(consultar_tensao_material("FoFo nodular"))
# Saída: 80

print(consultar_tensao_material("SAE 8620"))
# Saída: 200


80.0
200.0


In [17]:
# @title Tabela AGMA

# Substitua pelo seu ID real da planilha
sheet_id = "1oD-ugKvYCNAVJXnOAn-SwyeQto2QKA6pRr6565umGdQ"
sheet_name = "dados_servico"  # ou "Sheet1", "Planilha1" etc.

# Gera o link de exportação CSV
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carrega no DataFrame
df_servico = pd.read_csv(csv_url, decimal=',')

## Exibe Dataframe
# from IPython.display import display
# display(df_servico)

def consultar_fator_servico(aplicacao: str, horas: int = 10):
    """
    Consulta o fator de serviço da aplicação, com correspondência parcial e segura.
    """
    linha = df_servico[df_servico["Aplicacao"].str.contains(aplicacao, case=False, na=False, regex=False)]
    if linha.empty:
        return None

    saida = linha.iloc[0]["Servico_10h"] if horas == 10 else linha.iloc[0]["Servico_24h"]
    return float(saida)

fator_de_servico = consultar_fator_servico("Transportadores (esteira e correia)", horas=10)
# print(fator_de_servico)


In [18]:
# @title  Materiais e faixas de dureza Brinell
# Criando o DataFrame com os materiais e faixas de dureza Brinell

sheet_id = "1GmkBQOGUkKs6RmJMdjNqkbrkxzXkzLpPP5s2Znt4gRo"
sheet_name = "tabDurezaBrinell"  # ou "Sheet1", "Planilha1" etc.

# Gera o link de exportação CSV
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carrega no DataFrame, especificando dtypes and decimal separator
df_brinell_materiais = pd.read_csv(csv_url)
# from IPython.display import display
# display(df_brinell_materiais)
def consultar_faixa_brinell(material: str):
    """
    Retorna o intervalo de dureza Brinell (min e max) para o material informado,
    utilizando busca parcial e ignorando maiúsculas/minúsculas.
    """
    linha = df_brinell_materiais[df_brinell_materiais["Material"].str.contains(material, case=False, na=False)]
    if linha.empty:
        return f"Material '{material}' não encontrado."

    min_val = linha["Brinell_min"].iloc[0]
    max_val = linha["Brinell_max"].iloc[0]
    return min_val, max_val

_,a=consultar_faixa_brinell("Aço SAE 8620")
print(a)

2.600,00


In [19]:
# @title Conversão tipo de Motor
# Exibir o DataFrame
import pandas as pd
from IPython.display import display
# Criando o DataFrame com os valores da Tabela 2
df_conversao = pd.DataFrame({
    "Fator_Original": [1.00, 1.25, 1.75],
    "Motor_Eletrico_3h": [0.50, 1.00, 1.50],
    "Motor_Explosao_3h": [1.00, 1.25, 1.75],
    "Motor_Explosao_10h": [1.25, 1.50, 2.00],
    "Motor_Explosao_24h": [1.50, 1.75, 2.25]
})

display(df_conversao)

# Função de consulta
def converter_fator_servico(fator_original: float, tipo_servico: str):
    """
    tipo_servico pode ser:
        - "Motor_Eletrico_3h"
        - "Motor_Explosao_3h"
        - "Motor_Explosao_10h"
        - "Motor_Explosao_24h"
    """
    if tipo_servico not in df_conversao.columns:
        return f"Tipo de serviço '{tipo_servico}' inválido."

    linha = df_conversao[df_conversao["Fator_Original"] == fator_original]
    if linha.empty:
        return f"Fator de serviço original '{fator_original}' não encontrado na tabela."

    return linha.iloc[0][tipo_servico]
print(converter_fator_servico(1.25, "Motor_Explosao_10h"))

,Fator_Original,Motor_Eletrico_3h,Motor_Explosao_3h,Motor_Explosao_10h,Motor_Explosao_24h
0,1.00,0.5,1.00,1.25,1.50
1,1.25,1.0,1.25,1.50,1.75
2,1.75,1.5,1.75,2.00,2.25


1.5


In [20]:
# @title Função para Exibir Características Geométricas da Engrenagens de Dentes Retos
import pandas as pd
import numpy as np
from IPython.display import display

# Lista com grandeza, função lambda, fórmula simbólica
dados = [
    ["Módulo normalizado", lambda mn, Z: mn, "Módulo DIN 780"],
    ["Número de dentes", lambda mn, Z: Z, "Z"],
    ["Passo", lambda mn, Z: mn * np.pi, "t0 = mn * π"],
    ["Vão entre os dentes", lambda mn, Z: (mn * np.pi) / 2, "le = t0 / 2"],
    ["Altura da cabeça do dente", lambda mn, Z: mn, "hk = mn"],
    ["Altura do pé do dente", lambda mn, Z: 1.2 * mn, "hf = 1.2 * mn"],
    ["Altura comum do dente", lambda mn, Z: 2 * mn, "h = 2 * mn"],
    ["Altura total do dente", lambda mn, Z: 2.2 * mn, "ht = 2.2 * mn"],
    ["Espessura no primitivo", lambda mn, Z: (mn * np.pi) / 2, "S0 = t0 / 2"],
    ["Folga da cabeça", lambda mn, Z: 0.2 * mn, "Sk = 0.2 * mn"],
    ["Diâmetro primitivo", lambda mn, Z: mn * Z, "d = mn * Z"],
    ["Diâmetro de base", lambda mn, Z: (mn * Z) * np.cos(np.deg2rad(20)), "db = d * cos(α)"],
    ["Diâmetro interno", lambda mn, Z: (mn * Z) - 2.4 * mn, "di = d - 2.4 * mn"],
    ["Diâmetro externo", lambda mn, Z: (mn * Z) + 2 * mn, "da = d + 2 * mn"],
    ["Distância entre os centros", lambda d01, d02: (d01 + d02) / 2, "Cc = ( d01 + d02 ) / 2"],
    ["Largura das Engrenagens", lambda b0: b0, "b = b0"],
]

# Criar DataFrame com colunas base
df_dimensoes = pd.DataFrame(dados, columns=["Grandeza", "Funcao", "Formula"])
df_dimensoes["Args_Pinhão"] = None
df_dimensoes["Args_Coroa"] = None

# Função que calcula os argumentos extras para as funções específicas
def preencher_dimensoes_completo(mn_p, Z_p, mn_c, Z_c, b0):
    # Cálculos intermediários
    d_p = mn_p * Z_p
    d_c = mn_c * Z_c
    b_p = b0
    b_c = b0

    # Definir os argumentos necessários para cada linha
    df_dimensoes["Args_Pinhão"] = [
        (mn_p, Z_p),  (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p),
        (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p),
        (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p), (mn_p, Z_p),
        (mn_p, Z_p), (d_p, d_c), (b_p,)
    ]
    df_dimensoes["Args_Coroa"] = [
        (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c),
        (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c),
        (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c), (mn_c, Z_c),
        (mn_c, Z_c), (d_p, d_c), (b_c,)
    ]

    # Aplicar as funções usando os argumentos específicos
    df_dimensoes["Pinhão (mm)"] = df_dimensoes.apply(
        lambda row: round(row["Funcao"](*row["Args_Pinhão"]), 3), axis=1
    )
    df_dimensoes["Coroa (mm)"] = df_dimensoes.apply(
        lambda row: round(row["Funcao"](*row["Args_Coroa"]), 3), axis=1
    )

    return df_dimensoes[["Grandeza", "Formula", "Pinhão (mm)", "Coroa (mm)"]]

# Exemplo de uso
resultado = preencher_dimensoes_completo(2.25, 29, 2.25, 110,23)
display(resultado)


,Grandeza,Formula,Pinhão (mm),Coroa (mm)
0,Módulo normalizado,Módulo DIN 780,2.250,2.250
1,Número de dentes,Z,29.000,110.000
2,Passo,t0 = mn * π,7.069,7.069
3,Vão entre os dentes,le = t0 / 2,3.534,3.534
4,Altura da cabeça do dente,hk = mn,2.250,2.250
5,Altura do pé do dente,hf = 1.2 * mn,2.700,2.700
6,Altura comum do dente,h = 2 * mn,4.500,4.500
7,Altura total do dente,ht = 2.2 * mn,4.950,4.950
8,Espessura no primitivo,S0 = t0 / 2,3.534,3.534
9,Folga da cabeça,Sk = 0.2 * mn,0.450,0.450


In [21]:
# @title Função para Exibir Dimensionamento
def exibe_dimensionamento(Mt, W, Padm, X, d01, ms, ms_norm, d01_recalculo, b1, tau_max, tau_mat, redimensionar):
  df = pd.DataFrame({
          "Grandeza": [
              "Momento torçor (Mt)",
              "Fator de durabilidade (W)",
              "Pressão admissível (Padm)",
              "Volume mínimo (X)",
              "Diâmetro primitivo estimado (d01)",
              "Módulo calculado (ms)",
              "Módulo normalizado (ms_norm)",
              "Diâmetro primitivo recalculado (d01)",
              "Largura do pinhão (b1)",
              "Tensão de flexão (τmax)",
              "Tensão admissível (τadm)",
              "Necessita redimensionar?"
          ],
          "Valor": [
              round(Mt, 2),
              round(W, 2),
              round(Padm, 2),
              round(X, 2),
              round(d01, 2),
              round(ms, 2),
              round(ms_norm, 2),
              round(d01_recalculo, 2),
              round(b1, 2),
              round(tau_max, 2),
              round(tau_mat, 2),
              "Sim" if not redimensionar else "Não"
          ],
          "Unidade": [
              "N.mm", "", "MPa", "mm³", "mm", "", "", "mm", "mm", "MPa", "MPa", ""
          ]
      })

  return df

In [22]:
# Exibir o DataFrame
import pandas as pd


# @title Dimensionamento de Engrenagens de Dentes Retos


# @markdown ### Dados do Motor:
# @markdown ##### Potência (kw):
P_kW = 5.5 # @param {"type":"number","placeholder":"11"}

# @markdown ##### Rotação (rpm):
n_rpm = 706.43  # @param {"type":"number","placeholder":"11"}    # Rotação (rpm)
# @markdown ##### Relação de transmissão:
i = 2.52 # @param {"type":"number","placeholder":"11"} # Relação de transmissão

# @markdown ### Dados da Engrenagem:
# @markdown ##### Número de dentes do pinhão
Z1 = 27 # @param {"type":"number","placeholder":"11"}  # Número de dentes do pinhão

# @markdown ##### Ângulo de pressao (graus)
Angulo_de_Pressao = 20 # @param {"type":"number","placeholder":"11"} # Ângulo de pressao (graus)

# @markdown ##### Total de horas estimadas de durabilidade
h = 15000 # @param {"type":"number","placeholder":"11"}  # Horas de funcionamento

# @markdown ##### Fator de serviço tabela AGMA (consulte a tabela acima)
Tipo_aplicação = "Eixo de Transmissão - Cargas uniformes" # @param ["Agitadores - Líquidos", "Agitadores - Misturadores de polpa", "Agitadores - Semilíquidos densidade variável", "Alimentadores helicoidais", "Alimentadores recíprocos", "Transportadores (esteira e correia)", "Bombas - Centrífugas", "Bombas - Dupla ação multicilíndrica", "Bombas - Recíprocas de descargas livres", "Bombas - Rotativas de engrenagens ou lobos", "Britadores - Pedra e minérios", "Cervejarias - Cozinheiros serviço contínuo", "Cervejarias - Tachos de fermentação", "Clarificadores", "Classificadores", "Dragas - Guinchos, transportadores e bombas", "Dragas - Cabeçotes rotativos e peneiras", "Eixo de Transmissão - Cargas uniformes", "Eixo de Transmissão - Cargas pesadas", "Elevadores - Caçambas uniformes", "Elevadores - Caçambas carga pesada", "Elevadores de carga", "Embobinadeiras - Metais", "Embobinadeiras - Papel", "Embobinadeiras - Têxtil", "Enlatadoras e Engarrafadoras", "Escadas rolantes", "Fábrica de cimento - Britadores de mandíbulas", "Fábrica de cimento - Fornos rotativos", "Fábrica de cimento - Moinhos de bolas e rolos", "Fábrica de papel - Agitadores", "Fábrica de papel - Alvejadores", "Fábrica de papel - Batedores e despolpadores", "Fábrica de papel - Calandras", "Fábrica de papel - Hipercalandras", "Fábrica de papel - Cilindros", "Descascadores mecânicos e hidráulicos", "Tambores e descascadores", "Embobinadeiras", "Esticadores de feltro", "Jardanas", "Prensas", "Secadoras", "Geradores", "Guinchos e Gruas - Cargas uniformes", "Guinchos e Gruas - Cargas pesadas", "Indústria Alimentícia - Cozinhadores de cereais", "Indústria Alimentícia - Enlatadoras e Engarrafadoras", "Indústria Alimentícia - Misturadores de massa", "Indústria Alimentícia - Moedores de carne", "Indústria Alimentícia - Picadores", "Indústria Borracha e Plástico - Calandras", "Indústria Borracha e Plástico - Equipamentos de laboratório", "Indústria Borracha e Plástico - Extrusoras", "Indústria Borracha e Plástico - Moinhos", "Indústria Borracha e Plástico - Moinhos cilíndricos", "Indústria Borracha e Plástico - 2 em linha", "Indústria Borracha e Plástico - 3 em linha", "Indústria Borracha e Plástico - Refinadores", "Indústria Borracha e Plástico - Trituradores e misturadores", "Indústria Madeireira - Alimentadores de plaina", "Indústria Madeireira - Serras", "Indústria Madeireira - Tombadores", "Indústria Madeireira - Transportadores de tora", "Indústria Têxtil - Calandras", "Indústria Têxtil - Cordas", "Indústria Têxtil - Fiação", "Indústria Têxtil - Retorcedeiras", "Indústria Têxtil - Máquinas de tinturaria", "Indústria Têxtil - Fielas e rebobinadeiras", "Indústria Metalúrgica - Cortadores de chapa", "Máquinas operatrizes - Acionamento principal (pesadas)", "Máquinas operatrizes - Acionamento principal (uniformes)", "Máquinas operatrizes - Acionamento auxiliar", "Máquinas operatrizes - Prensas", "Misturadores - Betoneiras", "Misturadores - Líquidos de densidade constante", "Misturadores - Líquidos de densidade variável", "Misturadores - Líquidos para borracha", "Misturadores - Líquidos para polpa de painel", "Moinhos - Bolas e rolos", "Moinhos - Martelos", "Moinhos - Areia", "Olaria e Cerâmica - Extrusoras e misturadores", "Olaria"]
# @markdown ###### Marque/Desmarque para 10h/24h:
tempo_aplicacao24h = False # @param {"type":"boolean","placeholder":"true"}

# @markdown ##### Relação largura / diâmetro primitivo
Y = 0.25 # @param {"type":"number","placeholder":"11"}  # Relação largura / diâmetro primitivo

# @markdown ##### Selecione o Material de Fabricação da Engrenage:
Material = "SAE 8640" # @param ["FoFo cinzento","FoFo nodular","Aço fundido","SAE 1010", "SAE 1020","SAE 1040", "SAE 1050","SAE 4320", "SAE 4340","SAE 8620", "SAE 8640","Mat. Sintético - Resinas"]
# @markdown ###### Marque/Desmarque para usar valor Max/Min de dureza do material:
UsarDurezaMaxima = True # @param {"type":"boolean","placeholder":"true"}

# @markdown ##### Dureza do Material HB:

HB = 60 # @param {"type":"number","placeholder":"11"}  # Relação largura / diâmetro primitivo
UnidadeDureza = "Rockwell_C" # @param ["Brinell_Impressao_mm",	"Dureza_HB",	"Resistencia_Nmm2",	"Rockwell_C",	"Rockwell_B",	"Rockwell_A",	"Shore",	"Vickers"]
# @markdown ###### Marque/Desmarque para usar valor Max/Min de dureza do material:
UsarDurezaInformada = True # @param {"type":"boolean","placeholder":"true"}

# @markdown ###### Marque/Desmarque para engrenamento interno/externo:
eng_interno = True # @param {"type":"boolean","placeholder":"true"}
# @markdown ###### Marque/Desmarque para engrenamento biapoiado/em balanço:
biapoiado = True # @param {"type":"boolean","placeholder":"true"}

engrenamento_interno = 1 if eng_interno else -1

tipo_engrenamento = 1.2 if biapoiado else 0.75

modulos_normais = np.array([1.0, 1.25, 1.5, 1.75, 2.0, 2.25,
                            2.5, 2.75, 3.0, 3.25, 3.5, 3.75,
                            4.0, 4.5, 5, 5.5, 6, 6.5, 7, 8, 9,
                            10, 11, 12, 13, 14, 15, 16, 18,
                            20, 22, 24, 27, 30, 33, 36, 39, 42, 45,
                            50, 55, 60, 65, 70, 75])

tau_mat = consultar_tensao_material(Material) # Tensão do material

if UsarDurezaInformada:
  HB = converter_dureza(HB, f'{UnidadeDureza}',"Dureza_HB")

  if not HB:
    HB = consultar_faixa_brinell(Material)[1] if UsarDurezaMaxima else consultar_faixa_brinell(Material)[0]

else:
  HB = consultar_faixa_brinell(Material)[1] if UsarDurezaMaxima else consultar_faixa_brinell(Material)[0]
  # print(f'HB={HB}')

phi = consultar_fator_servico(f'{Tipo_aplicação}', 24 if tempo_aplicacao24h else 10)

# ---------- Cálculos ----------
beta_rad = np.radians(Angulo_de_Pressao)

# 1. Momento torçor
Mt = (30000 * P_kW *1000) / (np.pi * n_rpm)

# 2. Fator de durabilidade
W = (60 * n_rpm * h) / 1e6

# 3. Pressão admissível
Padm = (0.487 * HB) / (W**(1./6.))

# 4. Volume mínimo
X = 5.72E5 * phi * Mt / (Padm**2)  * (i + engrenamento_interno) /(i+ 0.14 * engrenamento_interno)

# 5. Diâmetro primitivo estimado
d01 = (X / Y) ** (1/3)

# 6. Módulo da engrenagem
ms = d01 / Z1

#normalizar para o valor mais proximo da tabela DIN 780 pag 99/100 Melconian
#indice_do_menor_modulo = np.argmin(np.abs(modulos_normais - ms))
# ms_norm = modulos_normais[indice_do_menor_modulo]

modulos_filtrados = modulos_normais[modulos_normais >= ms]
if np.size(modulos_filtrados)>0:
  ms_norm = np.min(modulos_filtrados)
else:
  ms_norm = np.max(modulos_normais)


indice_do_menor_modulo = np.argmin(modulos_normais == ms_norm)
print(indice_do_menor_modulo)
# 7. Diametro primitivo recalculado
d01_recalculo = ms_norm * Z1

# 8. Largura do pinhão
b1 = np.ceil(X / d01_recalculo ** 2)

# 9. Tensão Máxima de Flexão no Pé do Dente
Ft = 2 * Mt / d01_recalculo
q = interpolar_q(Z1, 'interno' if tipo_engrenamento == -1 else 'externo')      # obtido da tabela na pagina 102 em função de numero de dentes

tau_max = Ft * q * phi / (b1 * ms_norm)

# 10. Analise de dimensionamento
# print(tau_mat)
# print(tau_max)

redimensionar = tau_max > tau_mat

# ---------- Resultados em tabela ----------

df_final = pd.DataFrame({
    "Descrição": [
        # Dados de entrada
        "Potência do motor (P_kW)",
        "Rotação do motor (n_rpm)",
        "Relação de transmissão (i)",
        "Número de dentes do pinhão (Z1)",
        "Ângulo de pressão (graus)",
        "Horas de funcionamento (h)",
        "Tipo de aplicação",
        "Aplicação 24h?",
        "Relação largura / diâmetro (Y)",
        "Material da engrenagem",
        "Usar dureza máxima?",
        "Engrenamento interno?",
        "Engrenamento biapoiado?",
        "Dureza Brinell (HB)",

        # Resultados calculados
        "Momento torçor (Mt)",
        "Fator de durabilidade (W)",
        "Pressão admissível (Padm)",
        "Volume mínimo necessário (X)",
        "Diâmetro primitivo estimado (d01)",
        "Módulo frontal calculado (ms)",
        "Módulo normalizado escolhido (ms_norm)",
        "Diâmetro primitivo recalculado (d01_final)",
        "Largura da engrenagem (b1)",
        "Força tangencial (Ft)",
        "Tensão máxima no pé do dente (tau_max)",
        "Tensão admissível do material (tau_mat)",
        "Redimensionar? (tau_max > tau_mat)"
    ],
    "Valor": [
        # Dados de entrada
        P_kW,
        n_rpm,
        i,
        Z1,
        Angulo_de_Pressao,
        h,
        Tipo_aplicação,
        "Sim" if tempo_aplicacao24h else "Não",
        Y,
        Material,
        "Sim" if UsarDurezaMaxima else "Não",
        "Sim" if eng_interno else "Não",
        "Sim" if biapoiado else "Não",
        HB,

        # Resultados calculados
        Mt,
        W,
        Padm,
        X,
        d01,
        ms,
        ms_norm,
        d01_recalculo,
        b1,
        Ft,
        tau_max,
        tau_mat,
        "Sim" if redimensionar else "Não"
    ],
    "Unidade": [
        "kW",
        "rpm",
        "-",
        "",
        "°",
        "h",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "HB",
        "Nm",
        "-",
        "N/mm²",
        "mm³",
        "mm",
        "mm",
        "mm",
        "mm",
        "mm",
        "N",
        "N/mm²",
        "N/mm²",
        ""
    ]
})

# Exibir no Colab
# display.display(df_final)

# # Exibição no Colab
import IPython.display as display
display.display(df_final)


if redimensionar:
  print("Usando a hipotese 1 ...")

  # ---------- Resultados ----------
  # 1º hipotese: calculando a largura do pinhão via tensão do material
  b1H1 = 0
  if redimensionar:
      b1H1 = Ft * q * phi / (tau_mat * ms_norm)

  # verificando relação largura b e diametro primitivo d0 atende ao tipo de engrenamento

  teste_hipo1 = b1H1 / d01_recalculo

  if not teste_hipo1 < tipo_engrenamento:
    hipotese1 = False
  else:
    hipotese1 = True


  d01_final = ms_norm * Z1
  b1H1 = np.round(b1H1,0)


  # Exemplo de uso
  if hipotese1:
    resultado = preencher_dimensoes_completo(ms_norm, Z1,ms_norm, np.round(Z1*i,0),b1H1)
  else:
    print(">>Hipotese 1 falhou")

  # Exibição no Colab
  import IPython.display as display
  display.display(resultado)

  # 2º hipótese: aumentando o valor do módulo normalizado
  print("Usando a hipotese 2 ...")
  indice = indice_do_menor_modulo+1;

  while indice <= np.size(modulos_normais)-1:

    ms_norm = modulos_normais[indice]

    d01_recalculo = ms_norm * Z1

    Ft = 2 * Mt / d01_recalculo

    tau_max = Ft * q * phi / (b1 * ms_norm)

    if tau_max < tau_mat:
      break

    indice += 1


  # verificando relação largura b e diametro primitivo d0 atende ao tipo de engrenamento

  teste_hipo2 = b1 / d01_recalculo

  if not teste_hipo1 < tipo_engrenamento:
    hipotese2 = False
  else:
    hipotese2 = True


  # Exemplo de uso
  if hipotese2:
    # ---------- Resultados em tabela ----------
    print("Hipotese 2: Aumentar o valor do módulo normalizado")
    print(ms_norm)
    print(b1)
    print(d01_recalculo)
    print(tau_max)
    print(tau_mat)

    resultado = preencher_dimensoes_completo(ms_norm, Z1,ms_norm, np.round(Z1*i,0),b1)
  else:
    print(">>Hipotese 2 falhou")


  # resultado = preencher_dimensoes_completo(ms_norm, Z1,ms_norm, np.round(Z1*i,0),np.round(b1,0))

  # Exibir o DataFrame
  # Exibição no Colab
  import IPython.display as display
  display.display(resultado)




0


,Descrição,Valor,Unidade
0,Potência do motor (P_kW),5.5,kW
1,Rotação do motor (n_rpm),706.43,rpm
2,Relação de transmissão (i),2.52,-
3,Número de dentes do pinhão (Z1),27,
4,Ângulo de pressão (graus),20,°
5,Horas de funcionamento (h),15000,h
6,Tipo de aplicação,Eixo de Transmissão - Cargas uniformes,
7,Aplicação 24h?,Não,
8,Relação largura / diâmetro (Y),0.25,
9,Material da engrenagem,SAE 8640,


Usando a hipotese 1 ...


,Grandeza,Formula,Pinhão (mm),Coroa (mm)
0,Módulo normalizado,Módulo DIN 780,2.250,2.250
1,Número de dentes,Z,27.000,68.000
2,Passo,t0 = mn * π,7.069,7.069
3,Vão entre os dentes,le = t0 / 2,3.534,3.534
4,Altura da cabeça do dente,hk = mn,2.250,2.250
5,Altura do pé do dente,hf = 1.2 * mn,2.700,2.700
6,Altura comum do dente,h = 2 * mn,4.500,4.500
7,Altura total do dente,ht = 2.2 * mn,4.950,4.950
8,Espessura no primitivo,S0 = t0 / 2,3.534,3.534
9,Folga da cabeça,Sk = 0.2 * mn,0.450,0.450


Usando a hipotese 2 ...
Hipotese 2: Aumentar o valor do módulo normalizado
2.5
15.0
67.5
183.5734686837834
200.0


,Grandeza,Formula,Pinhão (mm),Coroa (mm)
0,Módulo normalizado,Módulo DIN 780,2.500,2.500
1,Número de dentes,Z,27.000,68.000
2,Passo,t0 = mn * π,7.854,7.854
3,Vão entre os dentes,le = t0 / 2,3.927,3.927
4,Altura da cabeça do dente,hk = mn,2.500,2.500
5,Altura do pé do dente,hf = 1.2 * mn,3.000,3.000
6,Altura comum do dente,h = 2 * mn,5.000,5.000
7,Altura total do dente,ht = 2.2 * mn,5.500,5.500
8,Espessura no primitivo,S0 = t0 / 2,3.927,3.927
9,Folga da cabeça,Sk = 0.2 * mn,0.500,0.500


In [ ]:
# @title Dimensionamento de Engrenagens de Dentes Helicoidais

In [65]:
# @title Função para Exibir Características Geométricas da Engrenagens de Dentes Helicoidais
import pandas as pd
import numpy as np
from IPython.display import display

# Lista com grandeza, função lambda, fórmula simbólica
dadosHeloidal = [
    ["Número de dentes", lambda Z: Z, "Z"],
    ["Módulo normalizado", lambda mn, Z: mn, "Módulo DIN 780"],
    ["Passo", lambda mn, Z: mn * np.pi, "t0 = mn * π"],
    ["Vão entre os dentes", lambda mn, Z: (mn * np.pi) / 2, "le = t0 / 2"],
    ["Espessura no primitivo", lambda mn, Z: (mn * np.pi) / 2, "S0 = t0 / 2"],
    ["Altura da cabeça do dente", lambda mn, Z: mn, "hk = mn"],
    ["Altura do pé do dente", lambda mn, Z: 1.2 * mn, "hf = 1.2 * mn"],
    ["Altura total do dente", lambda mn, Z: 2.2 * mn, "ht = 2.2 * mn"],
    ["Altura comum do dente", lambda mn, Z: 2 * mn, "h = 2 * mn"],
    ["Folga da cabeça do dente", lambda mn, Z: 0.2 * mn, "Sk = 0.2 * mn"],
    ["Módulo Frontal", lambda ms: ms, "ms = mn / cos alfa"],
    ["Angulo de Pressão Frontal (alpha)", lambda alpha, beta0: np.degrees(np.arctan(np.tan(alpha) / np.cos(beta0))), "tg alfa = tg alfa_ns / cos beta0"],
    ["Avanço do dente", lambda b, beta0: b * np.tan(beta0), "s = b * tan beta0"],
    ["Diâmetro primitivo", lambda ms, Z:  Z * ms, "d0 = ms * Z"],
    ["Diâmetro de base", lambda ms, Z, alpha: (ms * Z) * np.cos(alpha), "db = d * cos(α)"],
    ["Diâmetro interno", lambda ms, Z, mn: (ms * Z) - 2.4 * mn, "di = d - 2.4 * mn"],
    ["Diâmetro externo", lambda ms, Z, mn: (ms * Z) + 2 * mn, "da = d + 2 * mn"],
    ["Distância entre os centros", lambda d01, d02: (d01 + d02) / 2, "Cc = ( d01 + d02 ) / 2"],
    ["Largura das Engrenagens", lambda b0: b0, "b = b0"],
]

# Criar DataFrame com colunas base
df_dimensoesHelicoidal = pd.DataFrame(dadosHeloidal, columns=["Grandeza", "Funcao", "Formula"])
df_dimensoesHelicoidal["Args_Pinhão"] = None
df_dimensoesHelicoidal["Args_Coroa"] = None

# Função que calcula os argumentos extras para as funções específicas
def preencher_dimensoes_completo_Helicoidal(mn, Z_p, Z_c, b0, alfa_n, beta0):
    # Cálculos intermediários
    ms_p = mn / np.cos(beta0)
    ms_c = mn / np.cos(beta0)
    d_p = ms_p * Z_p
    d_c = ms_c * Z_c
    b_p = b0
    b_c = b0
    alfa_p = np.arctan(np.tan(alfa_n)/ np.cos(beta0))
    alfa_c = np.arctan(np.tan(alfa_n)/ np.cos(beta0))


    # Definir os argumentos necessários para cada linha as tuples
    df_dimensoesHelicoidal["Args_Pinhão"] = [
        (Z_p,),  (mn, Z_p), (mn, Z_p), (mn, Z_p), (mn, Z_p),
        (mn, Z_p), (mn, Z_p), (mn, Z_p), (mn, Z_p), (mn, Z_p),
        (ms_p,),(alfa_n,beta0), (b_p,beta0), (ms_p, Z_p),
        (ms_p, Z_p,alfa_p), (ms_p, Z_p, mn),(ms_p, Z_p, mn), (d_p, d_c), (b_p,)
    ]
    df_dimensoesHelicoidal["Args_Coroa"] = [
        (Z_c,), (mn, Z_c), (mn, Z_c), (mn, Z_c), (mn, Z_c),
        (mn, Z_c), (mn, Z_c), (mn, Z_c), (mn, Z_c), (mn, Z_c),
        (ms_c,),(alfa_n,beta0), (b_c,beta0), (ms_c, Z_c),
        (ms_c, Z_c, alfa_c), (ms_c, Z_c, mn),(ms_c, Z_c, mn), (d_p, d_c), (b_c,)
    ]

    # Aplicar as funções usando os argumentos específicos
    df_dimensoesHelicoidal["Pinhão (mm)"] = df_dimensoesHelicoidal.apply(
        lambda row: round(row["Funcao"](*row["Args_Pinhão"]), 3), axis=1
    )
    df_dimensoesHelicoidal["Coroa (mm)"] = df_dimensoesHelicoidal.apply(
        lambda row: round(row["Funcao"](*row["Args_Coroa"]), 3), axis=1
    )

    return df_dimensoesHelicoidal[["Grandeza", "Formula", "Pinhão (mm)", "Coroa (mm)"]]

# Exemplo de uso
# resultadoHelicoidal = preencher_dimensoes_completo_Helicoidal(2.25, 29, 110,23 , 20, 25)
# display(resultadoHelicoidal)

In [66]:
# @title Tabelas φp vs β0
# Tabela φp vs β0 em um DataFrame e funções utilitárias

import pandas as pd
import numpy as np

# Datagrama (tabela) como DataFrame
phi_p_table = pd.DataFrame({
    "beta_deg": [0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
    "phi_p":    [1.00, 1.11, 1.22, 1.31, 1.40, 1.47, 1.54, 1.60, 1.66, 1.71],
}).set_index("beta_deg").astype(float)

def phi_p_interpolada(beta_0: float) -> float:
    """
    Retorna φp para um ângulo β0 (graus).
    - Se β0 estiver entre dois pontos da tabela, faz interpolação linear.
    - Se β0 coincidir com a tabela, retorna o valor exato.
    Levanta ValueError se estiver fora do intervalo [0, 45].
    """
    beta_0 = float(beta_0)
    x = phi_p_table.index.to_numpy(dtype=float)
    y = phi_p_table["phi_p"].to_numpy(dtype=float)
    if beta_0 < x.min() or beta_0 > x.max():
        raise ValueError(f"β0={beta_0} fora do intervalo [{x.min()}, {x.max()}].")
    return float(np.interp(beta_0, x, y))

def phi_p_arredondando_para_cima(beta_0: float) -> float:
    """
    Retorna φp usando o próximo ângulo da tabela >= β0 (sem interpolar).
    Útil quando se quer projeto conservador ('sempre para cima').
    """
    beta_0 = float(beta_0)
    idx = phi_p_table.index
    if beta_0 < idx.min() or beta_0 > idx.max():
        raise ValueError(f"β0={beta_0} fora do intervalo [{idx.min()}, {idx.max()}].")
    proximo = idx[idx >= beta_0].min()
    return float(phi_p_table.loc[proximo, "phi_p"])

# Demonstração rápida:
# print("φp(20°) interpolada =", phi_p_interpolada(20))     # exato
# print("φp(12°) interpolada =", phi_p_interpolada(12))     # entre 10° e 15°
# print("φp(12°) arredonda↑  =", phi_p_arredondando_para_cima(12))  # usa 15°



In [24]:
import pandas as pd
# @title Fator de Elasticidade
# Criando o DataFrame diretamente
dados_fator_f = [
    ["Aço", "Aço", 1512],
    ["Aço", "FoFo", 1234],
    ["FoFo", "FoFo", 1069]
]

df_fator_f = pd.DataFrame(dados_fator_f, columns=["Material_Pinhao", "Material_Coroa", "Fator_f"])

# Função para obter o fator f dado os dois materiais
def obter_fator_f(material_pinhao, material_coroa):
    filtro = (
        (df_fator_f["Material_Pinhao"].str.lower() == material_pinhao.lower()) &
        (df_fator_f["Material_Coroa"].str.lower() == material_coroa.lower())
    )
    resultado = df_fator_f.loc[filtro, "Fator_f"]
    if resultado.empty:
        raise ValueError("Combinação de materiais não encontrada na tabela.")
    return resultado.values[0]

# Exemplo de uso:
print(obter_fator_f("Aço", "Aço"))   # 1512
print(obter_fator_f("Aço", "FoFo"))  # 1234
print(obter_fator_f("FoFo", "FoFo")) # 1069


1512
1234
1069


In [25]:
# @title Fator de Correção de Hélice
import pandas as pd

# Criando DataFrame da Tabela 5 - Fator de correção da hélice (Resistência)
dados_phir = {
    "beta0_min": [0,   5,   10,   15,   25],  # limite inferior da faixa
    "beta0_max": [5,   10,   15,   25,   46],  # limite superior da faixa
    "phi_r":     [1.00, 1.2, 1.28, 1.35, 1.36]
}

df_phir = pd.DataFrame(dados_phir)

# Função para obter φᵣ sem interpolação, apenas por faixa
def obter_phi_r(beta0):
    """
    Retorna o valor de φᵣ (fator de correção da hélice - resistência)
    com base na tabela, sem interpolação, apenas por faixa de valores.

    beta0: Ângulo da hélice em graus
    """
    linha = df_phir[(beta0 >= df_phir["beta0_min"]) & (beta0 < df_phir["beta0_max"])]
    if not linha.empty:
        return linha.iloc[0]["phi_r"]
    else:
        raise ValueError(f"Ângulo β₀={beta0}° fora da faixa da tabela.")

# Exemplo de uso
print(df_phir)
print("phi_r para β₀=12°:", obter_phi_r(12))
print("phi_r para β₀=28°:", obter_phi_r(20))


   beta0_min  beta0_max  phi_r
0          0          5   1.00
1          5         10   1.20
2         10         15   1.28
3         15         25   1.35
4         25         46   1.36
phi_r para β₀=12°: 1.28
phi_r para β₀=28°: 1.35


In [64]:
# Exibir o DataFrame
import pandas as pd


# @title Dimensionamento de Engrenagens de Dentes Helicoidais


# @markdown ### Dados do Motor:
# @markdown ##### Potência (kw):
P_kW = 3 # @param {"type":"number","placeholder":"11"}

# @markdown ##### Rotação (rpm):
n_rpm = 1730  # @param {"type":"number","placeholder":"11"}    # Rotação (rpm)

# @markdown ### Dados da Engrenagem:
# @markdown ##### Número de dentes do pinhão
Z1 = 21 # @param {"type":"number","placeholder":"11"}  # Número de dentes do pinhão
# @markdown ##### Número de dentes do pinhão
Z2 = 75 # @param {"type":"number","placeholder":"11"}  # Número de dentes do pinhão


# @markdown ##### Ângulo de Pressao Normal (graus)
Angulo_de_Pressao_Normal = 20 # @param {"type":"number","placeholder":"11"} # Ângulo de pressao (graus)

# @markdown ##### Ângulo de Hélice (graus)
Angulo_de_Helice = 20 # @param {"type":"number","placeholder":"11"} # Ângulo de pressao (graus)

# @markdown ##### Total de horas estimadas de durabilidade
h = 12000 # @param {"type":"number","placeholder":"11"}  # Horas de funcionamento

# @markdown ##### Fator de serviço tabela AGMA (consulte a tabela acima)
Tipo_aplicação = "Eixo de Transmissão - Cargas uniformes" # @param ["Agitadores - Líquidos", "Agitadores - Misturadores de polpa", "Agitadores - Semilíquidos densidade variável", "Alimentadores helicoidais", "Alimentadores recíprocos", "Transportadores (esteira e correia)", "Bombas - Centrífugas", "Bombas - Dupla ação multicilíndrica", "Bombas - Recíprocas de descargas livres", "Bombas - Rotativas de engrenagens ou lobos", "Britadores - Pedra e minérios", "Cervejarias - Cozinheiros serviço contínuo", "Cervejarias - Tachos de fermentação", "Clarificadores", "Classificadores", "Dragas - Guinchos, transportadores e bombas", "Dragas - Cabeçotes rotativos e peneiras", "Eixo de Transmissão - Cargas uniformes", "Eixo de Transmissão - Cargas pesadas", "Elevadores - Caçambas uniformes", "Elevadores - Caçambas carga pesada", "Elevadores de carga", "Embobinadeiras - Metais", "Embobinadeiras - Papel", "Embobinadeiras - Têxtil", "Enlatadoras e Engarrafadoras", "Escadas rolantes", "Fábrica de cimento - Britadores de mandíbulas", "Fábrica de cimento - Fornos rotativos", "Fábrica de cimento - Moinhos de bolas e rolos", "Fábrica de papel - Agitadores", "Fábrica de papel - Alvejadores", "Fábrica de papel - Batedores e despolpadores", "Fábrica de papel - Calandras", "Fábrica de papel - Hipercalandras", "Fábrica de papel - Cilindros", "Descascadores mecânicos e hidráulicos", "Tambores e descascadores", "Embobinadeiras", "Esticadores de feltro", "Jardanas", "Prensas", "Secadoras", "Geradores", "Guinchos e Gruas - Cargas uniformes", "Guinchos e Gruas - Cargas pesadas", "Indústria Alimentícia - Cozinhadores de cereais", "Indústria Alimentícia - Enlatadoras e Engarrafadoras", "Indústria Alimentícia - Misturadores de massa", "Indústria Alimentícia - Moedores de carne", "Indústria Alimentícia - Picadores", "Indústria Borracha e Plástico - Calandras", "Indústria Borracha e Plástico - Equipamentos de laboratório", "Indústria Borracha e Plástico - Extrusoras", "Indústria Borracha e Plástico - Moinhos", "Indústria Borracha e Plástico - Moinhos cilíndricos", "Indústria Borracha e Plástico - 2 em linha", "Indústria Borracha e Plástico - 3 em linha", "Indústria Borracha e Plástico - Refinadores", "Indústria Borracha e Plástico - Trituradores e misturadores", "Indústria Madeireira - Alimentadores de plaina", "Indústria Madeireira - Serras", "Indústria Madeireira - Tombadores", "Indústria Madeireira - Transportadores de tora", "Indústria Têxtil - Calandras", "Indústria Têxtil - Cordas", "Indústria Têxtil - Fiação", "Indústria Têxtil - Retorcedeiras", "Indústria Têxtil - Máquinas de tinturaria", "Indústria Têxtil - Fielas e rebobinadeiras", "Indústria Metalúrgica - Cortadores de chapa", "Máquinas operatrizes - Acionamento principal (pesadas)", "Máquinas operatrizes - Acionamento principal (uniformes)", "Máquinas operatrizes - Acionamento auxiliar", "Máquinas operatrizes - Prensas", "Misturadores - Betoneiras", "Misturadores - Líquidos de densidade constante", "Misturadores - Líquidos de densidade variável", "Misturadores - Líquidos para borracha", "Misturadores - Líquidos para polpa de painel", "Moinhos - Bolas e rolos", "Moinhos - Martelos", "Moinhos - Areia", "Olaria e Cerâmica - Extrusoras e misturadores", "Olaria"]
# @markdown ###### Marque/Desmarque para 10h/24h:
tempo_aplicacao24h = False # @param {"type":"boolean","placeholder":"true"}

# @markdown ##### Relação largura / diâmetro primitivo
Y = 0.25 # @param {"type":"number","placeholder":"11"}  # Relação largura / diâmetro primitivo

# @markdown ##### Selecione o Material de Fabricação da Engrenage:
Material = "SAE 8640" # @param ["FoFo cinzento","FoFo nodular","Aço fundido","SAE 1010", "SAE 1020","SAE 1040", "SAE 1050","SAE 4320", "SAE 4340","SAE 8620", "SAE 8640","Mat. Sintético - Resinas"]
# @markdown ###### Marque/Desmarque para usar valor Max/Min de dureza do material:
UsarDurezaMaxima = True # @param {"type":"boolean","placeholder":"true"}

# @markdown ##### Dureza do Material HB:

HB = 60 # @param {"type":"number","placeholder":"11"}  # Relação largura / diâmetro primitivo
UnidadeDureza = "Rockwell_C" # @param ["Brinell_Impressao_mm",	"Dureza_HB",	"Resistencia_Nmm2",	"Rockwell_C",	"Rockwell_B",	"Rockwell_A",	"Shore",	"Vickers"]
# @markdown ###### Marque/Desmarque para usar valor Max/Min de dureza do material:
UsarDurezaInformada = True # @param {"type":"boolean","placeholder":"true"}

# @markdown ###### Marque/Desmarque para engrenamento interno/externo:
eng_interno = True # @param {"type":"boolean","placeholder":"true"}
# @markdown ###### Marque/Desmarque para engrenamento biapoiado/em balanço:
biapoiado = True # @param {"type":"boolean","placeholder":"true"}

engrenamento_interno = 1 if eng_interno else -1

tipo_engrenamento = 1.2 if biapoiado else 0.75

modulos_normais = np.array([1.0, 1.25, 1.5, 1.75, 2.0, 2.25,
                            2.5, 2.75, 3.0, 3.25, 3.5, 3.75,
                            4.0, 4.5, 5, 5.5, 6, 6.5, 7, 8, 9,
                            10, 11, 12, 13, 14, 15, 16, 18,
                            20, 22, 24, 27, 30, 33, 36, 39, 42, 45,
                            50, 55, 60, 65, 70, 75])

# Relação de transmissão:
i = Z2/Z1

tau_mat = consultar_tensao_material(Material) # Tensão do material

if UsarDurezaInformada:
  HB = converter_dureza(HB, f'{UnidadeDureza}',"Dureza_HB")

  if not HB:
    HB = consultar_faixa_brinell(Material)[1] if UsarDurezaMaxima else consultar_faixa_brinell(Material)[0]

else:
  HB = consultar_faixa_brinell(Material)[1] if UsarDurezaMaxima else consultar_faixa_brinell(Material)[0]
  # print(f'HB={HB}')

phi = consultar_fator_servico(f'{Tipo_aplicação}', 24 if tempo_aplicacao24h else 10)

# ---------- Cálculos ----------
beta_rad = np.radians(Angulo_de_Pressao_Normal)

alfa_norm = np.radians(Angulo_de_Pressao_Normal)

beta0 = np.radians(Angulo_de_Helice)

f = obter_fator_f("Aço", "Aço")

e = 1 / phi #fator de serviço
phir = obter_phi_r(Angulo_de_Helice)
# 1. Momento torçor
Mt = (30000 * P_kW * 1000) / (np.pi * n_rpm)

# 2. Fator de durabilidade
W = (60 * n_rpm * h) / 1e6

# 3. Pressão admissível
Padm = (0.487 * HB) / (W**(1./6.))

# 4. Volume mínimo
f = obter_fator_f("Aço", "Aço")
phip = phi_p_interpolada(np.degrees(beta_rad))

X = 0.2 * f**2 * Mt / (phip * Padm**2)  * (i + 1) /i

# 5. Diâmetro primitivo estimado
d01 = (X / Y) ** (1/3)

# 6. Módulo da engrenagem
ms = d01 / Z1

# modulo frontal ferramenta

mn = ms * np.cos(beta0)

#normalizar para o valor mais proximo da tabela DIN 780 pag 99/100 Melconian
#indice_do_menor_modulo = np.argmin(np.abs(modulos_normais - ms))
# ms_norm = modulos_normais[indice_do_menor_modulo]

modulos_filtrados = modulos_normais[modulos_normais >= mn]
if np.size(modulos_filtrados)>0:
  ms_norm = np.min(modulos_filtrados)
else:
  ms_norm = np.max(modulos_normais)

indice_do_menor_modulo = np.argmin(modulos_normais == ms_norm)

# recalculo do módulo frontal
ms0 = ms_norm / np.cos(beta0)


# 7. Diametro primitivo recalculado
d01_recalculo = ms0 * Z1

# 8. Largura do pinhão
b1 = np.ceil(X / d01_recalculo ** 2)

# 9. Tensão Máxima de Flexão no Pé do Dente
Ft = 2 * Mt / d01_recalculo

Ze = Z1 / (np.cos(beta0)**3 )
q = interpolar_q(Ze, 'interno' if tipo_engrenamento == -1 else 'externo')      # obtido da tabela na pagina 102 em função de numero de dentes

e = 1 / phi
tau_max = Ft * q / (b1 * ms_norm * e * phir)

# 10. Analise de dimensionamento
# print(tau_mat)
# print(tau_max)

redimensionar = tau_max > tau_mat

# ---------- Resultados em tabela ----------

df_final = pd.DataFrame({
    "Descrição": [
        # Dados de entrada
        "Potência do motor (P_kW)",
        "Rotação do motor (n_rpm)",
        "Relação de transmissão (i)",
        "Número de dentes do pinhão (Z1)",
        "Número de dentes do pinhão (Z2)",
        "Ângulo de pressão (graus)",
        "Ângulo de Hélice (graus)",
        "Horas de funcionamento (h)",
        "Tipo de aplicação",
        "Aplicação 24h?",
        "Relação largura / diâmetro (Y)",
        "Material da engrenagem",
        "Usar dureza máxima?",
        "Engrenamento interno?",
        "Engrenamento biapoiado?",
        "Dureza Brinell (HB)",

        # Parametros
        "Fator de serviço (phi)",
        "Fator de correção da hélice (phir)",
        "Fator de correção da pressão (phip)",
        "Fator de atrito (q)",

        # Resultados calculados
        "Momento torçor (Mt)",
        "Fator de durabilidade (W)",
        "Pressão admissível (Padm)",
        "Volume mínimo necessário (X)",
        "Diâmetro primitivo estimado (d01)",
        "Módulo frontal calculado (ms)",
        "Módulo normalizado escolhido (ms_norm)",
        "Módulo frontal calculado (ms0)",
        "Diâmetro primitivo recalculado (d01_final)",
        "Número de dentes (Ze)",
        "Largura da engrenagem (b1)",
        "Força tangencial (Ft)",
        "Tensão máxima no pé do dente (tau_max)",
        "Tensão admissível do material (tau_mat)",
        "Redimensionar? (tau_max > tau_mat)"
    ],
    "Valor": [
        # Dados de entrada
        P_kW,
        n_rpm,
        i,
        Z1,
        Z2,
        Angulo_de_Pressao,
        Angulo_de_Helice,
        h,
        Tipo_aplicação,
        "Sim" if tempo_aplicacao24h else "Não",
        Y,
        Material,
        "Sim" if UsarDurezaMaxima else "Não",
        "Sim" if eng_interno else "Não",
        "Sim" if biapoiado else "Não",
        HB,
        # Parametros
        phi,
        phir,
        phip,
        q,

        # Resultados calculados
        Mt,
        W,
        Padm,
        X,
        d01,
        ms,
        ms_norm,
        ms0,
        d01_recalculo,
        Ze,
        b1,
        Ft,
        tau_max,
        tau_mat,
        "Sim" if redimensionar else "Não"
    ],
    "Unidade": [
        "kW",
        "rpm",
        "-",
        "",
        "",
        "°",
        "°",
        "h",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "HB",
        "",
        "",
        "",
        "Nm",
        "-",
        "N/mm²",
        "mm³",
        "mm",
        "mm",
        "mm",
        "mm",
        "mm",
        "dentes",
        "mm",
        "N",
        "N/mm²",
        "N/mm²",
        ""
    ]
})

# Exibir no Colab
# display.display(df_final)

# # Exibição no Colab
import IPython.display as display
display.display(df_final)


if redimensionar:
  print("Usando a hipotese 1 ...")

  # ---------- Resultados ----------
  # 1º hipotese: calculando a largura do pinhão via tensão do material
  b1H1 = 0
  if redimensionar:
      b1H1 = Ft * q / (tau_mat * ms_norm * e * phir)

  # verificando relação largura b e diametro primitivo d0 atende ao tipo de engrenamento

  teste_hipo1 = b1H1 / d01_recalculo

  if not teste_hipo1 < tipo_engrenamento:
    hipotese1 = False
  else:
    hipotese1 = True


  d01_final = ms_norm * Z1
  b1H1 = np.round(b1H1,0)


  # Exemplo de uso
  if hipotese1:
    resultado = preencher_dimensoes_completo_Helicoidal(ms_norm, Z1, Z2,b1H1,alfa_norm,beta0)
  else:
    print(">>Hipotese 1 falhou")

  # Exibição no Colab
  import IPython.display as display
  display.display(resultado)

  # 2º hipótese: aumentando o valor do módulo normalizado
  print("Usando a hipotese 2 ...")
  indice = indice_do_menor_modulo+1;

  while indice <= np.size(modulos_normais)-1:

    ms_norm = modulos_normais[indice]

    d01_recalculo = ms_norm * Z1

    Ft = 2 * Mt / d01_recalculo

    tau_max = Ft * q * phi / (b1 * ms_norm)

    if tau_max < tau_mat:
      break

    indice += 1


  # verificando relação largura b e diametro primitivo d0 atende ao tipo de engrenamento

  teste_hipo2 = b1 / d01_recalculo

  if not teste_hipo1 < tipo_engrenamento:
    hipotese2 = False
  else:
    hipotese2 = True


  # Exemplo de uso
  if hipotese2:
    # ---------- Resultados em tabela ----------
    print("Hipotese 2: Aumentar o valor do módulo normalizado")
    print(ms_norm)
    print(b1)
    print(d01_recalculo)
    print(tau_max)
    print(tau_mat)
    resultado = preencher_dimensoes_completo_Helicoidal(ms_norm, Z1, Z2,b1H1,alfa_norm,beta0)

    # resultado = preencher_dimensoes_completo_Helicoidal(ms_norm, Z1,ms_norm, np.round(Z1*i,0),b1)
  else:
    print(">>Hipotese 2 falhou")


  # resultado = preencher_dimensoes_completo(ms_norm, Z1,ms_norm, np.round(Z1*i,0),np.round(b1,0))

  # Exibir o DataFrame
  # Exibição no Colab
  import IPython.display as display
  display.display(resultado)

else:
  resultado = preencher_dimensoes_completo_Helicoidal(ms_norm, Z1, Z2,b1H1,alfa_norm,beta0)

  import IPython.display as display
  display.display(resultado)



,Descrição,Valor,Unidade
0,Potência do motor (P_kW),3,kW
1,Rotação do motor (n_rpm),1730,rpm
2,Relação de transmissão (i),3.571429,-
3,Número de dentes do pinhão (Z1),21,
4,Número de dentes do pinhão (Z2),75,
5,Ângulo de pressão (graus),20,°
6,Ângulo de Hélice (graus),20,°
7,Horas de funcionamento (h),12000,h
8,Tipo de aplicação,Eixo de Transmissão - Cargas uniformes,
9,Aplicação 24h?,Não,


,Grandeza,Formula,Pinhão (mm),Coroa (mm)
0,Número de dentes,Z,21.000,75.000
1,Módulo normalizado,Módulo DIN 780,1.500,1.500
2,Passo,t0 = mn * π,4.712,4.712
3,Vão entre os dentes,le = t0 / 2,2.356,2.356
4,Espessura no primitivo,S0 = t0 / 2,2.356,2.356
5,Altura da cabeça do dente,hk = mn,1.500,1.500
6,Altura do pé do dente,hf = 1.2 * mn,1.800,1.800
7,Altura total do dente,ht = 2.2 * mn,3.300,3.300
8,Altura comum do dente,h = 2 * mn,3.000,3.000
9,Folga da cabeça do dente,Sk = 0.2 * mn,0.300,0.300
